In [1]:
import pyspark
from currency_converter import CurrencyConverter, ECB_URL
from delta import *
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "10g") \
    .config("spark.driver.maxResultSize", "10g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/03/09 17:40:39 WARN Utils: Your hostname, Andreis-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 143.205.184.230 instead (on interface en0)
23/03/09 17:40:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/andreikirpichev/.ivy2/cache
The jars for the packages stored in: /Users/andreikirpichev/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4c367913-6d52-412e-a1ee-9fd365120595;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 137ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number

23/03/09 17:40:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 17:40:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [45]:
bm = spark.read.format("delta").load("/Users/andreikirpichev/parquet/betmarkets_lifespan/")
bm_by_outcome = bm.select('contest_id', 'section', 'point_type', 'market_type', 'market_params', 'away', 'end', 'home', 'league', col('odds').alias("odds_map"), 'region', 'sport', 'start', explode("odds").alias("outcome", "odds"))
bm_contest = bm_by_outcome.withColumn('contest_id', split(bm_by_outcome.contest_id, '\|').getItem(1))
bm_football = bm_contest.filter(bm_contest["sport"] == "Football")
# test: from 2022-10-18 09:32:32.232000  to  2022-10-28 14:23:44.972000 (974330, 13)
MAX_START = "2022-10-28 14:23:44.972000"
MIN_END = "2022-10-18 09:32:32.232000"
test_bm = bm_football.filter((bm_football["start"] <= MAX_START) &
                             (bm_football["end"] >= MIN_END))
test_bm = test_bm.select('contest_id', 'section', col('point_type').alias('point'), col('market_type').alias('market'), 'away', 'start', 'end', 'home', 'league', 'region', 'sport', col('odds').alias('placement_odds'))
#test_bm.write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/test_betmarkets")

ERROR:root:KeyboardInterrupt while sending command.                 (2 + 6) / 8]
Traceback (most recent call last):
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/andreikirpichev/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [44]:
bets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/bets/")
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/fixtures/")
betmarkets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/betmarkets/")

In [46]:
bets_df.filter(col("sport") != "Football").show()

+-----------+---+-------+-------+-----+-------------+-------------+----------+--------------+--------------------+--------------+--------------------+--------------------+------+---------------+--------------------+--------------------+---------+---------+-------------+----------+--------+---------+
|customer_id|age|country|section|point|       market|market_params|contest_id|placement_date|      placement_time|placement_odds|                home|                away|region|          sport|              league|minutes_before_start|stake_eur|dayofweek|weekday_stake|section_2w|point_2w|market_2w|
+-----------+---+-------+-------+-----+-------------+-------------+----------+--------------+--------------------+--------------+--------------------+--------------------+------+---------------+--------------------+--------------------+---------+---------+-------------+----------+--------+---------+
|    8718910| 41|     DE|     RT| Goal|   Handicap3w|         null|  IH-22533|    2022-09-23|2022

In [13]:
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/test_betmarkets/")
fixtures_df.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------+-----+--------------------+------+-----------+----+------------+--------------------+
|contest_id|                away|          duplicates|                home|              league|           linked_to|organisation|   receive_timestamp|region|round|     scheduled_begin|season|      sport|tier|trading_type|               venue|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+------+-----+--------------------+------+-----------+----+------------+--------------------+
|   1535058| Zhetysu Taldykorgan|                  []| Okzhetpes Kokshetau|      Premier League|[BB.Unibet|120718...|            |2020-11-15 12:36:...|    KZ|     | 2020-11-08 12:00:00|      |   Football|   6|        Live|                    |
|   4106585|Bayer 04 (Ma

In [25]:
betmarkets_df.schema

StructType([StructField('contest_id', ArrayType(StringType(), False), True), StructField('section', StringType(), True), StructField('point_type', StringType(), True), StructField('market_type', StringType(), True), StructField('market_params', MapType(StringType(), StringType(), True), True), StructField('certainty', DoubleType(), True), StructField('odds', MapType(StringType(), DoubleType(), True), True), StructField('receive_timestamp', TimestampType(), True), StructField('sources', MapType(StringType(), DoubleType(), True), True), StructField('status', StringType(), True), StructField('valid_from', TimestampType(), True)])

In [48]:
bets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/bet_stats/")
fixtures_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/fixtures/")
customers_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/customers/")

recent_dates = ['2022-10-02', '2022-10-03', '2022-10-04', '2022-10-05', '2022-10-06', '2022-10-07', '2022-10-08', '2022-10-09', '2022-10-10', '2022-10-11', '2022-10-12', '2022-10-13', '2022-10-14', '2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-22', '2022-10-23', '2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-29', '2022-10-30', '2022-10-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24', '2022-09-25', '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30', '2022-09-01', '2022-09-31']

bets_df = bets_df.filter(col("placement_date").isin(recent_dates))\
                 .withColumn('contest_id', split('contest_id', '\|').getItem(1))

bf = bets_df.join(fixtures_df, "contest_id", "left")
bf_football = bf.filter(col("sport") == "Football")

data = bf_football.join(customers_df, bf_football.customer_id == customers_df.id, "left")

columns = ['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'placement_date', 'placement_time', 'placement_odds', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'stake_eur']
# 'bet_no', 'pick_index', 'fees', 'outcomes', 'currency', 'stake', 'placement_time', 'betting_types', 'scheduled_begin', 'birthdate'
c = CurrencyConverter(ECB_URL, fallback_on_missing_rate=True)
convert_curr = udf(lambda v,f,d : c.convert(v, f, 'EUR', d), DoubleType())

weekday_until_now = Window.partitionBy("customer_id", "dayofweek").rowsBetween(Window.unboundedPreceding, Window.currentRow)
section_last2weeks = Window.partitionBy("customer_id", "section").orderBy(expr("unix_date(placement_date)").asc()).rangeBetween(-14, 0)
point_last2weeks = Window.partitionBy("customer_id", "point").orderBy(expr("unix_date(placement_date)").asc()).rangeBetween(-14, 0)
market_last2weeks = Window.partitionBy("customer_id", "market").orderBy(expr("unix_date(placement_date)").asc()).rangeBetween(-14, 0)

data.withColumn('age', ceil(datediff('placement_date', 'birthdate')/365.25))\
    .withColumn('minutes_before_start',
                  when(
                      round((unix_timestamp('scheduled_begin') - unix_timestamp('placement_time')) / 60) < 0,
                      0)
                  .otherwise(round((unix_timestamp('scheduled_begin') - unix_timestamp('placement_time')) / 60)))\
    .withColumn('section', split(bets_df['betting_types'].getItem(0), ' ').getItem(0))\
    .withColumn('point', split(bets_df['betting_types'].getItem(0), ' ').getItem(1))\
    .withColumn('market', split(bets_df['betting_types'].getItem(0), ' ').getItem(2))\
    .withColumn('market_params', regexp_extract(bets_df['betting_types'].getItem(1), '\(.*?\)', 0))\
    .filter(data['currency'] != 'NEO')\
    .withColumn('stake_eur', round(convert_curr('stake', 'currency', 'placement_date')/100, 2))\
    .select(*columns)\
    .withColumn("dayofweek", dayofweek(col("placement_date")))\
    .withColumn("weekday_stake", round(avg(col("stake_eur")).over(weekday_until_now), 2))\
    .withColumn("section_2w", round(avg(col("stake_eur")).over(section_last2weeks), 2))\
    .withColumn("point_2w", round(avg(col("stake_eur")).over(point_last2weeks), 2))\
    .withColumn("market_2w", round(avg(col("stake_eur")).over(market_last2weeks), 2))\
    .filter(col("age").isNotNull() & col("country").isNotNull())\
    .orderBy(col("placement_time").asc())\
    .write.format("delta").mode("overwrite").save("/Users/andreikirpichev/parquet/bets")


In [7]:
bets_df = spark.read.format("delta").load("/Users/andreikirpichev/parquet/bets/")

bets_df.select(countDistinct("market_params")).show()
bets_df.select([count(when(col(c).isNull(), c)).alias(c) for c in bets_df.columns]).show()

+-----------------------------+
|count(DISTINCT market_params)|
+-----------------------------+
|                         2320|
+-----------------------------+



+-----------+---+-------+-------+-----+------+-------------+----------+--------------+--------------+--------------+----+----+------+-----+------+--------------------+---------+---------+-------------+----------+--------+---------+
|customer_id|age|country|section|point|market|market_params|contest_id|placement_date|placement_time|placement_odds|home|away|region|sport|league|minutes_before_start|stake_eur|dayofweek|weekday_stake|section_2w|point_2w|market_2w|
+-----------+---+-------+-------+-----+------+-------------+----------+--------------+--------------+--------------+----+----+------+-----+------+--------------------+---------+---------+-------------+----------+--------+---------+
|          0|  0|      0|      0|    0|     0|      4684742|         0|             0|             0|             0|   0|   0|     0|    0|     0|                   0|        0|        0|            0|         0|       0|        0|
+-----------+---+-------+-------+-----+------+-------------+----------+-